In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import sqlite3
from datetime import date
from datetime import datetime as dt
import hashlib
import sys
import os
import locale
import re
import requests as rq
from contextlib import contextmanager

project_dir = '/home/mi/Dropbox/prog/python/tradeDB'
os.chdir(project_dir)

from importlib import reload
from workers import sql
from workers import api_stooq
from workers.common import read_json
from workers.common import get_cookie
import tradeDB

In [3]:
SECTORS = {
            "INDEXES": {"file": "./assets/indexes.json"},
            "STOCK": {"file": "./assets/stock.json"},
            "ETF": {"file": "./assets/etf.json"},
            "COMODITIES": {"file": "./assets/comodities.json"},
        }
for k in SECTORS:
    SECTORS[k]["data"] = read_json(
        SECTORS[k]["file"]
    )

In [16]:
reload(api_stooq)
#region = "EUROPE & CENTRAL ASIA"
#region = "U.S."
#region = "CANADA"
#region = "U.S. Main"
#region="LATIN AMERICA & CARIBBEAN"
region = "MIDDLE EAST & NORTH AFRICA"
api = SECTORS["INDEXES"]["data"][region]["api"]  # type: ignore
dat = api_stooq.web_stooq(
    sector_id=api["id"],  # type: ignore
    sector_grp=api["group"],  # type: ignore
    symbol="%",
    from_date=date.today(),
    end_date=date.today(),
)
dat.to_csv("./dev/tab6.csv")

In [43]:
dat=pd.read_csv("./dev/WIG.csv")
tab = "INDEXES"
db = './trader.sqlite'
names=dat['name']

In [27]:
tr=tradeDB.Trader()

In [39]:
s='WIG'
c='PL'
description={"country": c, "indexes": s}

In [29]:
datComp = api_stooq.get(
                        components=s,
                        from_date=date.today(),
                        end_date=date.today(),
                    )

In [31]:
dat=datComp
tab="STOCK"
dat["country"] = description["country"]
dat["tab"] = tab
dat["hash"] = [
    hashlib.md5("".join(r).encode("utf-8")).hexdigest()
    for r in dat.loc[:, ["symbol", "name", "tab"]].to_records(index=False)
]
dat.drop(columns=["tab"], inplace=True)

In [42]:
reload(sql)
t='STOCK_DESC'
sql_columns = sql.tab_columns(t, tr.db)
resp = sql.write_table(
    dat=dat.loc[:, [c in sql_columns for c in dat.columns]],
    tab=t,
    db_file=tr.db,
)

PRAGMA foreign_keys = ON
pragma table_info(STOCK_DESC)
PRAGMA foreign_keys = ON
INSERT OR REPLACE INTO STOCK_DESC ('symbol', 'name', 'country', 'hash', 'from_date', 'to_date')
            VALUES ('06N', '06MAGNA', 'PL', '1fe3990855a72d75b22c05a1c7421261', '2023-03-07', '2023-03-07')
        
INSERT OR REPLACE INTO STOCK_DESC ('symbol', 'name', 'country', 'hash', 'from_date', 'to_date')
            VALUES ('11B', '11BIT', 'PL', '3e96bd5ace285206f8ec9060ed2514ed', '2023-03-07', '2023-03-07')
        
INSERT OR REPLACE INTO STOCK_DESC ('symbol', 'name', 'country', 'hash', 'from_date', 'to_date')
            VALUES ('1AT', 'ATAL', 'PL', 'ace198b9fd8f659c618c329d16f85630', '2023-03-07', '2023-03-07')
        
INSERT OR REPLACE INTO STOCK_DESC ('symbol', 'name', 'country', 'hash', 'from_date', 'to_date')
            VALUES ('3RG', '3RGAMES', 'PL', '7fc47350880cd0ab6183d005f7ce4816', '2023-03-07', '2023-03-07')
        
INSERT OR REPLACE INTO STOCK_DESC ('symbol', 'name', 'country', 'hash', '

In [48]:
dates = pd.Series(['15:34','Feb 6', '7 Lut', '24 Feb 2023'])
dates

0          15:34
1          Feb 6
2          7 Lut
3    24 Feb 2023
dtype: object

In [38]:
@contextmanager
def setlocale(*args, **kwargs):
    # temporary change locale
    saved = locale.setlocale(locale.LC_ALL)
    yield locale.setlocale(*args, **kwargs)
    locale.setlocale(locale.LC_ALL, saved)

def date_locale(date: str, local: str, format: str):
    with setlocale(locale.LC_ALL, local):  # type: ignore
        return pd.to_datetime(date, errors='coerce',format=format)
        


In [49]:
d1 = pd.to_datetime(dates, errors="coerce")  # hh:ss
d1

0   2023-03-09 15:34:00
1                   NaT
2                   NaT
3   2023-02-24 00:00:00
dtype: datetime64[ns]

In [47]:
d2 = dates.apply(lambda x: date_locale(x, "en_GB.utf8", "%b %d"))  # 2022 22 Lut
d2

0          NaT
1   1900-02-06
2          NaT
dtype: datetime64[ns]

In [41]:
d3 = dates.apply(lambda x: date_locale(x, "pl_PL.utf8", "%Y %d %b"))  # 2022 22 Lut
d3

0          NaT
1          NaT
2   2023-02-07
dtype: datetime64[ns]

In [43]:
d1 = d1.fillna(d2)
d1 = d1.fillna(d3)
d1 = d1.fillna("")
d1

0   2023-03-09 15:34:00
1   2022-02-06 00:00:00
2   2023-02-07 00:00:00
dtype: datetime64[ns]